## Setting up LFW dataset

In [7]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz -q

In [8]:
!tar -xf lfw.tgz -q

#### Optionally remove exceedingly frequent examples

In [ ]:
# !cd lfw && du --inodes -S | sort -rh | sed -n '1,50{/^.\{71\}/s/^\(.\{30\}\).*\(.\{37\}\)$/\1...\2/;p}' > f
# !cd lfw && sed -i 's/.*\.\/\(.*\)/\1/' f
# !mkdir tmp
# !cd lfw && xargs mv -t ../tmp < f
# !rm lfw/f

#### Create zip archive for LFW dataset

In [9]:
!cd lfw && zip -r -q ../lfw.zip *

## SphereFace

In [4]:
!git clone --depth=1 https://github.com/clcarwin/sphereface_pytorch sphereface -q

In [5]:
!sudo apt install atool -y -qq
!cd sphereface/model && atool -x -q *.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 14999693 bytes (15 MiB)

Extracting archive: sphere20a_20171020.7z
--
Path = sphere20a_20171020.7z
Type = 7z
Physical Size = 14999693
Headers Size = 146
Method = LZMA:25
Solid = -
Blocks = 1

  0%     18% - sphere20a_20171020.pth                              37% - sphere20a_20171020.pth                              59% - sphere20a_20171020.pth                              82% - sphere20a_20171020.pth                             Everything is Ok

Size:       112344519
Compressed: 1

In [ ]:
!python3 lfw_eval.py --model sphereface/model/sphere20a_20171020.pth --lfw 'lfw.zip'

## Deepface package

In [6]:
!pip3 install deepface -qq

     |████████████████████████████████| 51kB 1.6MB/s eta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


#### [`deepface`](https://github.com/serengil/deepface/) package provides VGG-Face , Google FaceNet, OpenFace, Facebook DeepFace and DeepID<br>
The helper script using `deepface` package is [DeepFace.py](DeepFace.py).

In [ ]:
!python3 lfw_eval.py --net $MODEL_NAME --lfw 'lfw.zip'

Replace $MODEL_NAME with one of `deepface`, `openface`, and `facenet`.

## Setting up FFHQ dataset

In [2]:
!wget https://archive.org/download/flicker_faceial_recognition/flickr.tar -q

In [12]:
!atool -x flickr.tar -q
!rm flickr.tar

flickr.tar: extracted to `thumbnails128x128'


## Setting up YouTube faces dataset

In [10]:
!wget https://archive.org/download/youtube_face_rec/youtube.tar -q

In [11]:
!atool -x youtube.tar -q
!rm youtube.tar

youtube.tar: extracted to `youv1'


### EDA

In [177]:
import numpy as np
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import imageio
from IPython.display import Image, display
from glob import glob
from collections import defaultdict
import random

In [179]:
random.randint(0,9999999)

7713907

In [73]:
file_list = sorted(glob('youv1/*'))

In [87]:
database = defaultdict(list)
for f in file_list:
    if f[-4:] == '.npz' and len(database[f.split('/')[1].split('.')[0][:-2]]) < 2:
        database[f.split('/')[1].split('.')[0][:-2]].append(f)
[k for k, v in database.items() if len(v) < 2]
for k in database.keys():
    if len(database[k]) < 2:
        database.pop(k)

In [126]:
x = np.load(database[list(database.keys())[0]][0])

In [127]:
list(x.keys())

['colorImages', 'boundingBox', 'landmarks2D', 'landmarks3D']

In [128]:
x['colorImages'].shape

(192, 125, 3, 73)

In [129]:
def norm(x):
    y = (x-x.mean())/x.std()
    y -= y.min()
    return y/y.max()

In [190]:
def tv(x):
    imgs = []
    for i in range(x['colorImages'].shape[-1]):
        im = x['colorImages'][..., i]
#         for coor in x['landmarks2D'][..., i]:
        for coor in av:
            coor = [int(x) for x in coor]
            for j in range(3):
                im[coor[1]-1:coor[1]+1, coor[0]-1:coor[0]+1, j] = im.max()
        imgs.append(norm(im))
    return imgs

In [191]:
imgs = tv(x)
print(len(imgs))
name = str(random.randint(0,9999999)) + '.gif'
imageio.mimsave(name, imgs, fps=24)

73


In [192]:
display(Image(url=name))

In [150]:
av = np.zeros((68,2))
n = 0
for v in database.values():
    for f in v:
        a = np.load(f)['landmarks2D']
        for i in range(a.shape[-1]):
            av += a[..., i]
            n += 1
av = av/n

In [151]:
n

61268

In [193]:
av.T

array([[ 20.26656656,  21.1005582 ,  23.15464843,  25.46972318,
         29.05898675,  34.86671672,  41.19116015,  49.27502122,
         61.74502187,  73.90835346,  81.88933864,  87.98509826,
         93.49918391,  96.46479402,  98.12920285,  99.55467781,
         99.78380231,  28.94963113,  33.42872299,  39.126167  ,
         44.41617157,  49.28512437,  68.8620487 ,  73.5006039 ,
         78.93437031,  84.90486061,  89.67531175,  59.40583992,
         59.54269766,  59.84309917,  60.1364497 ,  53.75096298,
         56.50350917,  60.29072273,  63.89072599,  66.65487693,
         36.54524385,  40.13284259,  44.90025788,  49.2772573 ,
         45.34930143,  40.29987922,  69.73020174,  74.12600379,
         78.90663968,  82.73828099,  78.84796305,  73.92137821,
         46.73384148,  51.36040021,  57.27761637,  60.43926683,
         63.4124339 ,  69.27240974,  73.8591924 ,  69.22346086,
         65.26062545,  60.71934778,  56.12522034,  51.72976105,
         47.6821832 ,  56.47155122,  60.